##Importing necessary libraries

In [105]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

##Data loading and preprocessing

In [106]:
file_paths = ['CAD and exercise.csv','CAD and exercise1.csv', 'CAD and exercise2.csv', 'CTG Alzeimers disease and exercise.csv', 'CTG Alzheimer disease and physical activity.csv', 'CTG Alzheimer disease.csv', 'CTG CAD and phycial activity.csv','CTG Cardiac and exercise.csv','CTG Cardio and exercise.csv','CTG Cardio and physical activity.csv', 'CTG Cardiovascular disease and exercise.csv', 'CTG Cardiovascular disease and physical activity.csv','CTG Chronic kidney disease and exercise.csv','CTG Chronic kidney disease and physical activity.csv','CTG Diabetes and exercise.csv','CTG Diabetes and physical activity.csv','CTG Diabetes mellitus and exercise.csv','CTG Diabetes mellitus type 2 and exercise.csv','CTG Diabetes mellitus type 2 and physical activity.csv', 'CTG Kneeosteoarthritis and exercise.csv','CTG Kneeosteoarthritis and physical activity.csv','CTG Obesity and exercise.csv','CTG Obesity and physical activity.csv','CTG Osteoarthritis and exercise.csv','CTG Osteoarthritis and physical activity.csv','CTG PAD and exercise 1.csv','CTG PAD and exercise.csv','CTG PAD and physical activity.csv','CTG PAD and physical activity1.csv','CTG Physical activity and CAD.csv','CTG heart and exercise.csv','CTG stroke and exercise.csv','CTG stroke and exercise1.csv','CTG stroke and physical activity.csv','CTG stroke and physical activity1.csv','CTG studies diabetes and physical activity.csv']

In [107]:

clinical_trial_dataframes = [pd.read_csv(file) for file in file_paths]
ct_data = pd.concat(clinical_trial_dataframes, ignore_index=True)

In [108]:
# Drop irrelevant columns
columns_to_drop = ['Study Title', 'Study URL', 'Acronym', 'Study Status',
                   'Brief Summary', 'Study Results', 'Sponsor', 'Collaborators',
                   'Phases', 'Funder Type', 'Study Type', 'Study Design', 'Start Date',
                   'Primary Completion Date', 'Completion Date', 'First Posted', 'Results First Posted',
                   'Last Update Posted', 'Locations', 'Study Documents']
ct_data.drop(columns=columns_to_drop, inplace=True)

In [109]:
ct_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1131 entries, 0 to 1130
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   NCT Number                  1080 non-null   object 
 1   Conditions                  1080 non-null   object 
 2   Interventions               1080 non-null   object 
 3   Primary Outcome Measures    1080 non-null   object 
 4   Secondary Outcome Measures  904 non-null    object 
 5   Other Outcome Measures      216 non-null    object 
 6   Sex                         1077 non-null   object 
 7   Age                         1080 non-null   object 
 8   Enrollment                  1080 non-null   float64
 9   Other IDs                   1080 non-null   object 
 10  Total number of studies     51 non-null     object 
 11  1158                        50 non-null     object 
 12  Unnamed: 2                  0 non-null      float64
 13  Unnamed: 3                  0 non

##handling missing values

In [110]:
ct_data.isnull().sum()


NCT Number                      51
Conditions                      51
Interventions                   51
Primary Outcome Measures        51
Secondary Outcome Measures     227
Other Outcome Measures         915
Sex                             54
Age                             51
Enrollment                      51
Other IDs                       51
Total number of studies       1080
1158                          1081
Unnamed: 2                    1131
Unnamed: 3                    1131
Type Intervention             1120
Unnamed: 5                    1120
Total number of patients      1121
dtype: int64

In [111]:
# Handle missing values
ct_data.dropna(subset=['Conditions', 'Interventions'], inplace=True)



In [113]:
conditions_counts = ct_data['Conditions'].value_counts().reset_index()
conditions_counts.columns = ['Condition', 'Total Count']
print(conditions_counts.to_string(index=False))


                                                                                                                                                                                                      Condition  Total Count
                                                                                                                                                                                                        Obesity           69
                                                                                                                                                                                      Diabetes Mellitus, Type 2           34
                                                                                                                                                                                                         Stroke           24
                                                                                                                    

In [114]:
Interventions_counts = ct_data['Interventions'].value_counts().reset_index()
Interventions_counts.columns = ['Interventions', 'Total Count']
print(Interventions_counts.to_string(index=False))

                                                                                                                                                                                                                                                                                                                                                                                                                                       Interventions  Total Count
                                                                                                                                                                                                                                                                                                                                                                                                                                BEHAVIORAL: Exercise           21
                                                                                                    

In [115]:
# Filter rows containing exercise-related interventions
exercise_data = ct_data[clinical_trials_data['Interventions'].str.contains('Exercise', case=False, na=False)]


<ipython-input-115-8f33ec17bb68>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  exercise_data = ct_data[clinical_trials_data['Interventions'].str.contains('Exercise', case=False, na=False)]


In [116]:
exercise_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 0 to 1062
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   NCT Number                  182 non-null    object 
 1   Conditions                  182 non-null    object 
 2   Interventions               182 non-null    object 
 3   Primary Outcome Measures    182 non-null    object 
 4   Secondary Outcome Measures  142 non-null    object 
 5   Other Outcome Measures      21 non-null     object 
 6   Sex                         182 non-null    object 
 7   Age                         182 non-null    object 
 8   Enrollment                  182 non-null    float64
 9   Other IDs                   182 non-null    object 
 10  Total number of studies     0 non-null      object 
 11  1158                        0 non-null      object 
 12  Unnamed: 2                  0 non-null      float64
 13  Unnamed: 3                  0 non-

##feature selection and encoding

In [117]:
# Split 'Conditions' column into individual conditions
conditions_split = exercise_data['Conditions'].str.split('|', expand=True)
conditions_split.columns = [f'Condition_{i}' for i in range(1, conditions_split.shape[1] + 1)]

# One-hot encode conditions
conditions_encoded = pd.get_dummies(conditions_split, prefix='Condition')

# Extract information about intervention types
intervention_types = exercise_data['Interventions'].str.split(':', n=1, expand=True)[0]
intervention_types_encoded = pd.get_dummies(intervention_types, prefix='Intervention')

# Encode sex and age into numerical values
sex_encoded = pd.get_dummies(exercise_data['Sex'], prefix='Sex')
age_encoded = pd.get_dummies(exercise_data['Age'], prefix='Age')

# Combine all encoded features
exercise_features = pd.concat([conditions_encoded, intervention_types_encoded, sex_encoded, age_encoded], axis=1)

# Display the engineered features
print(exercise_features.head())


    Condition_Aging  Condition_Alzheimer Disease 2 Due to Apoe4 Isoform  \
0                 0                                                  0    
3                 0                                                  1    
4                 0                                                  0    
7                 0                                                  0    
11                0                                                  0    

    Condition_Arthroplasty  Condition_Asthma in Children  \
0                        0                             0   
3                        0                             0   
4                        0                             0   
7                        0                             0   
11                       0                             0   

    Condition_Cardiovascular Diseases  Condition_Cardiovascular Risk Factor  \
0                                   0                                     0   
3                             

##Clustering

Apply a clustering algorithm (e.g., K-means) to group similar participants based on their characteristics and exercise-related features.
Determine the optimal number of clusters using techniques like the elbow method or silhouette analysis.
Assign cluster labels to each participant in the dataset.

In [118]:
from sklearn.cluster import KMeans

X = exercise_features

# Perform clustering
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X)

# Assign cluster labels to the exercise_data DataFrame
exercise_data['Cluster'] = clusters

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-118-bd904e7ad891>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exercise_data['Cluster'] = clusters


##Target Variable Creation

Analyze the characteristics and exercise patterns of each cluster.
Assign meaningful labels to each cluster based on their characteristics (e.g., "highly active," "moderately active," "sedentary").
Create a new target variable in the dataset based on the assigned cluster labels.

In [119]:
# Assign meaningful labels to clusters
cluster_labels = {
    0: 'Highly Active',
    1: 'Moderately Active',
    2: 'Sedentary'
}

exercise_data['Target'] = exercise_data['Cluster'].map(cluster_labels)

<ipython-input-119-f90af7e5dfd3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exercise_data['Target'] = exercise_data['Cluster'].map(cluster_labels)


In [120]:
exercise_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182 entries, 0 to 1062
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   NCT Number                  182 non-null    object 
 1   Conditions                  182 non-null    object 
 2   Interventions               182 non-null    object 
 3   Primary Outcome Measures    182 non-null    object 
 4   Secondary Outcome Measures  142 non-null    object 
 5   Other Outcome Measures      21 non-null     object 
 6   Sex                         182 non-null    object 
 7   Age                         182 non-null    object 
 8   Enrollment                  182 non-null    float64
 9   Other IDs                   182 non-null    object 
 10  Total number of studies     0 non-null      object 
 11  1158                        0 non-null      object 
 12  Unnamed: 2                  0 non-null      float64
 13  Unnamed: 3                  0 non-

##Model Training and Evaluation

Split the dataset into features (X) and the target variable (y).
Train a predictive model (e.g., Random Forest) using the cluster-based target variable.
Evaluate the model's performance using appropriate metrics and cross-validation techniques.

In [121]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming 'exercise_data' contains your dataset

# Separate features and target variable
X = exercise_data.drop(['Target'], axis=1)
y = exercise_data['Target']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocess categorical features using OneHotEncoder
categorical_features = X.select_dtypes(include=['object']).columns.tolist()
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(), categorical_features)
])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# Initialize and fit the RandomForestClassifier model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model using cross-validation and on the test set
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
print(f"Cross-validation scores: {cv_scores}")
print(f"Mean cross-validation score: {cv_scores.mean()}")

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Test set accuracy: {accuracy}")
print(classification_report(y_test, y_pred))

Cross-validation scores: [1. 1. 1. 1. 1.]
Mean cross-validation score: 1.0
Test set accuracy: 1.0
                   precision    recall  f1-score   support

    Highly Active       1.00      1.00      1.00         7
Moderately Active       1.00      1.00      1.00         9
        Sedentary       1.00      1.00      1.00        21

         accuracy                           1.00        37
        macro avg       1.00      1.00      1.00        37
     weighted avg       1.00      1.00      1.00        37



##Integration with the Exercise Habit App

Use the trained model to predict the exercise cluster for new app users based on their characteristics and preferences.
Personalize the app experience and provide exercise recommendations based on the predicted cluster.
Collect user feedback and exercise data to continuously improve the model and refine the clustering.

In [122]:
# Import necessary libraries
import pandas as pd

# Step 1: Enter demographic information
age = input("Enter your age: ")
sex = input("Enter your sex: ")
race = input("Enter your race: ")
gender = input("Enter your gender: ")
comorbidities = input("Enter any comorbidities (separated by commas if multiple): ").split(',')

# Step 2: Confirm weekly exercise goal
weekly_goal = input("Confirm your weekly exercise goal (e.g., 150 mins MVPA, plus strength training): ")

# Step 3: Preference
start_preference = input("Do you prefer a gradual start or an immediate goal? ")
exercise_variety = input("Do you prefer new types of exercise every day or consistency? ")

# Step 4: Location of exercise
exercise_location = input("Where do you plan to exercise (YMCA, Planet Fitness, other gym, home)? ")

# Step 5: Select days and times of exercise
days_of_exercise = input("Select the days of the week you plan to exercise (separated by commas if multiple): ").split(',')
times_of_exercise = input("Select the times of the day you plan to exercise (separated by commas if multiple): ").split(',')

# Step 6: Optional preference of exercise machine
exercise_machine_preference = input("Enter your preference of exercise machine (e.g., treadmill, elliptical): ")

# Step 7: Coordinate exercise buddy
exercise_buddy = input("Do you want to coordinate/have a potential exercise buddy? ")

# Step 8: Create appearance of virtual coach
# This step involves more complex graphics and is beyond the scope of this code snippet

# Displaying entered information
print("\nEntered information:")
print(f"Age: {age}")
print(f"Sex: {sex}")
print(f"Race: {race}")
print(f"Gender: {gender}")
print(f"Comorbidities: {comorbidities}")
print(f"Weekly exercise goal: {weekly_goal}")
print(f"Start preference: {start_preference}")
print(f"Exercise variety preference: {exercise_variety}")
print(f"Exercise location: {exercise_location}")
print(f"Days of exercise: {days_of_exercise}")
print(f"Times of exercise: {times_of_exercise}")
print(f"Exercise machine preference: {exercise_machine_preference}")
print(f"Exercise buddy coordination: {exercise_buddy}")


Enter your age: 27
Enter your sex: female
Enter your race: asian
Enter your gender: female
Enter any comorbidities (separated by commas if multiple): heart disease, diabetes
Confirm your weekly exercise goal (e.g., 150 mins MVPA, plus strength training): strength training
Do you prefer a gradual start or an immediate goal? gradual start
Do you prefer new types of exercise every day or consistency? every day
Where do you plan to exercise (YMCA, Planet Fitness, other gym, home)? home
Select the days of the week you plan to exercise (separated by commas if multiple): friday, saturday, sunday
Select the times of the day you plan to exercise (separated by commas if multiple): 1 
Enter your preference of exercise machine (e.g., treadmill, elliptical): elliptical
Do you want to coordinate/have a potential exercise buddy? yes

Entered information:
Age: 27
Sex: female
Race: asian
Gender: female
Comorbidities: ['heart disease', ' diabetes']
Weekly exercise goal: strength training
Start preferenc

In [71]:
import datetime
import time

# Step 1: Generate cardio and strength training exercises
# This step would require a database of exercises and routines, and is beyond the scope of this code snippet

# Step 2: User receives notification
notification_time = datetime.datetime.now() + datetime.timedelta(minutes=15)  # Notification 15 minutes before exercise session
print(f"Notification: Your exercise session will start at {notification_time.strftime('%H:%M')}.")

# Step 3: User starts recording exercise time
input("Press 'Enter' to begin recording exercise time.")

# Simulate exercise time recording
print("Exercise session started.")
time.sleep(120)  # Simulating 2 minutes of exercise time

# Step 4: User ends exercise session
input("Press 'Enter' to end exercise session.")

# Simulate exercise session end
print("Exercise session ended.")

# Step 5: Checkmark or X on types of exercises used
exercise_types = ['Cardio', 'Strength Training']  # List of exercise types
print("Types of exercises:")
for idx, exercise in enumerate(exercise_types):
    print(f"{idx+1}. {exercise}")

user_input = input("Enter the numbers of exercises used (separated by commas if multiple): ")
used_exercises = [exercise_types[int(i)-1] for i in user_input.split(',')]

# Step 6: Did they use another machine or not exercise?
if 'Strength Training' in used_exercises:
    additional_machine = input("Did you use another machine for strength training? (yes/no) ")
    if additional_machine.lower() == 'yes':
        additional_machine_used = input("Enter the name of the additional machine used: ")
        print(f"Additional machine used: {additional_machine_used}")
    else:
        print("No additional machine used.")
elif 'Cardio' in used_exercises:
    # If only cardio was used, we can ask if the user used any other cardio machine
    additional_machine = input("Did you use another cardio machine? (yes/no) ")
    if additional_machine.lower() == 'yes':
        additional_machine_used = input("Enter the name of the additional cardio machine used: ")
        print(f"Additional cardio machine used: {additional_machine_used}")
    else:
        print("No additional cardio machine used.")

# Step 7: Give reward if exercise time goal is met (gamification)
exercise_time_goal = 150  # Example exercise time goal in minutes
exercise_time = int(input("Enter the total exercise time (in minutes): "))
if exercise_time >= exercise_time_goal:
    print("Congratulations! You've met your exercise time goal.")
    # Provide some reward or feedback for achieving the goal
else:
    print("Keep up the good work! You'll reach your goal with consistency.")


Notification: Your exercise session will start at 20:44.
Press 'Enter' to begin recording exercise time.
Exercise session started.
Press 'Enter' to end exercise session.
Exercise session ended.
Types of exercises:
1. Cardio
2. Strength Training
Enter the numbers of exercises used (separated by commas if multiple): 2
Did you use another machine for strength training? (yes/no) yes
Enter the name of the additional machine used: treadmill
Additional machine used: treadmill
Enter the total exercise time (in minutes): 30
Keep up the good work! You'll reach your goal with consistency.


In [123]:
# Step 5: Integration with the Exercise Habit App
# Predict exercise cluster for the new user based on their characteristics and preferences
user_data = pd.DataFrame({
    'Age': [age],
    'Sex': [sex],
    'Race': [race],
    'Gender': [gender],
    'Comorbidities': [', '.join(comorbidities)],
    'Weekly_Goal': [weekly_goal],
    'Start_Preference': [start_preference],
    'Exercise_Variety': [exercise_variety],
    'Exercise_Location': [exercise_location],
    'Exercise_Machine_Preference': [exercise_machine_preference],
    'Exercise_Buddy': [exercise_buddy]
})


In [124]:
# Preprocess the user data
user_data_encoded = pd.get_dummies(user_data)


In [125]:
# Align the columns of user_data_encoded with the model's feature columns
missing_cols = set(X.columns) - set(user_data_encoded.columns)
for col in missing_cols:
    user_data_encoded[col] = 0
user_data_encoded = user_data_encoded[X.columns]

In [126]:
# Align the columns of user_data_encoded with the model's feature columns
missing_cols = set(X.columns) - set(user_data_encoded.columns)
for col in missing_cols:
    user_data_encoded[col] = 0
user_data_encoded = user_data_encoded[X.columns]


In [127]:
print("Features in X:")
print(X.columns)

print("\nFeatures in user_data_encoded:")
print(user_data_encoded.columns)

Features in X:
Index(['NCT Number', 'Conditions', 'Interventions', 'Primary Outcome Measures',
       'Secondary Outcome Measures', 'Other Outcome Measures', 'Sex', 'Age',
       'Enrollment', 'Other IDs', 'Total number of studies', '1158',
       'Unnamed: 2', 'Unnamed: 3', 'Type Intervention', 'Unnamed: 5',
       'Total number of patients', 'Cluster'],
      dtype='object')

Features in user_data_encoded:
Index(['NCT Number', 'Conditions', 'Interventions', 'Primary Outcome Measures',
       'Secondary Outcome Measures', 'Other Outcome Measures', 'Sex', 'Age',
       'Enrollment', 'Other IDs', 'Total number of studies', '1158',
       'Unnamed: 2', 'Unnamed: 3', 'Type Intervention', 'Unnamed: 5',
       'Total number of patients', 'Cluster'],
      dtype='object')


In [128]:
missing_cols = set(X.columns) - set(user_data_encoded.columns)
for col in missing_cols:
    user_data_encoded[col] = 0

In [129]:
user_data_encoded = user_data_encoded[X.columns]

In [130]:
import pandas as pd
from scipy.sparse import csr_matrix

# Convert X_train from csr_matrix to DataFrame
X_train_array = X_train.toarray()  # Convert csr_matrix to array
X_train_df = pd.DataFrame(X_train_array)

# Truncate columns if necessary
if X_train_array.shape[1] > len(X.columns):
    X_train_df = X_train_df.iloc[:, :len(X.columns)]
    X_train_df.columns = X.columns

# Create a new RandomForestClassifier model
new_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Fit the new model with feature names
new_model.fit(X_train_df, y_train)

# Predict the exercise cluster for the new user
user_cluster = new_model.predict(user_data_encoded)

In [131]:
# Personalize the app experience based on the predicted cluster
if user_cluster == 0:
    # Provide recommendations for highly active users
    print("Based on your profile, you are classified as a highly active user.")
    print("Here are some recommended exercises and routines for you:")
    # Provide personalized exercise recommendations for highly active users
    # ...

elif user_cluster == 1:
    # Provide recommendations for moderately active users
    print("Based on your profile, you are classified as a moderately active user.")
    print("Here are some recommended exercises and routines for you:")
    # Provide personalized exercise recommendations for moderately active users
    # ...

else:
    # Provide recommendations for sedentary users
    print("Based on your profile, you are classified as a sedentary user.")
    print("Here are some recommended exercises and routines for you:")
    # Provide personalized exercise recommendations for sedentary users
    # ...

Based on your profile, you are classified as a sedentary user.
Here are some recommended exercises and routines for you:


In [132]:
# Personalize the app experience based on the predicted cluster and user profile
if user_cluster == 0:
    # Provide recommendations for highly active users
    print("Based on your profile, you are classified as a highly active user.")
    print("Here are some recommended exercises and routines for you: Consider high-intensity interval training, strength training, and cardiovascular exercises. Adjust intensity based on individual health conditions")


elif user_cluster == 1:
    # Provide recommendations for moderately active users
    print("Based on your profile, you are classified as a moderately active user.")
    print("Here are some recommended exercises and routines for you: Include a mix of cardiovascular exercises, strength training, and flexibility exercises. Tailor workout plans to achieve fitness goals effectively")

else:
    # Provide recommendations for sedentary users
    print("Based on your profile, you are classified as a sedentary user.")
    print("Here are some recommended exercises and routines for you: Focus on low-impact activities like walking, yoga, or swimming to improve fitness gradually. Emphasize consistency and gradual progression in physical activity")


# Import necessary libraries and define user profiles and usage data as provided in the code snippet

# Further code implementation including decision tree model, linear regression, collaborative filtering, and knowledge-based systems can be integrated to enhance personalized exercise recommendations based on user data, preferences, and health considerations.

# Ensure to tailor exercise recommendations considering individual fitness levels, goals, age, gender, existing health conditions, and preferences to promote a safe and effective workout routine aligned with established guidelines.

# Continuously update and refine exercise recommendations based on user feedback, performance data, and evolving health guidelines to optimize the app experience and support users in achieving their fitness objectives effectively.

Based on your profile, you are classified as a sedentary user.
Here are some recommended exercises and routines for you: Focus on low-impact activities like walking, yoga, or swimming to improve fitness gradually. Emphasize consistency and gradual progression in physical activity


In [133]:
# Generate personalized exercise plan based on user's preferences
print("\nPersonalized Exercise Plan:")
print(f"Weekly Goal: {weekly_goal}")
print(f"Start Preference: {start_preference}")
print(f"Exercise Variety: {exercise_variety}")
print(f"Exercise Location: {exercise_location}")
print(f"Days of Exercise: {', '.join(days_of_exercise)}")
print(f"Times of Exercise: {', '.join(times_of_exercise)}")
print(f"Exercise Machine Preference: {exercise_machine_preference}")
print(f"Exercise Buddy: {exercise_buddy}")

# Provide personalized exercise recommendations based on the user's cluster and preferences
# ...

# Simulate exercise session
# ...

# Provide reward or feedback based on exercise performance
# ...


Personalized Exercise Plan:
Weekly Goal: strength training
Start Preference: gradual start
Exercise Variety: every day
Exercise Location: home
Days of Exercise: friday,  saturday,  sunday
Times of Exercise: 1 
Exercise Machine Preference: elliptical
Exercise Buddy: yes
